# FastAPI Security

**Big Picture of FastAPI Security**

- **Authentication**  
  Authentication is the process of verifying who a user is. In FastAPI, this can be handled using various methods such as OAuth2, API keys, JWT tokens, and others.

- **OAuth2**  
  OAuth2 is an authorization framework that enables applications to obtain limited access to user accounts without exposing their credentials. FastAPI has built-in support for OAuth2.


In [ ]:
from fastapi import Depends, HTTPException, status
from fastapi.security import OAuth2PasswordBearer

oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

async def get_current_user(token: str = Depends(oauth2_scheme)):
    # Replace this with your token validation logic
    if token != "mysecrettoken":
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Invalid authentication credentials",
            headers={"WWW-Authenticate": "Bearer"},
        )
    return {"username": "user"}


**JWT Tokens**  
JWT (JSON Web Token) is a compact, URL-safe means of representing claims between two parties. JWTs are often used for API authentication and server-to-server authorization.


In [ ]:
import jwt
from datetime import datetime, timedelta

SECRET_KEY = "your_secret_key"
ALGORITHM = "HS256"

def create_access_token(data: dict):
    to_encode = data.copy()
    expire = datetime.utcnow() + timedelta(minutes=30)
    to_encode.update({"exp": expire})
    encoded_jwt = jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)
    return encoded_jwt

### OpenID Connect - OIDC

**OpenID Connect (OIDC)**  
OIDC is an identity layer built on top of the OAuth 2.0 protocol. It allows clients to verify the identity of a user based on the authentication performed by an authorization server. In FastAPI, you can utilize OIDC for secure authentication.

**Step-by-Step Guide to Implement OIDC in FastAPI**

1. **Install Required Packages**  
   You'll need `fastapi`, `authlib`, and `httpx`.


In [ ]:
pip install fastapi authlib httpx uvicorn


2. **Configure OIDC Settings**  
   Define your OIDC settings such as the client ID, client secret, and the OIDC provider URL.


In [ ]:
# config.py

OIDC_CLIENT_ID = "your-client-id"
OIDC_CLIENT_SECRET = "your-client-secret"
OIDC_PROVIDER_URL = "https://accounts.google.com"  # Example for Google
OIDC_REDIRECT_URI = "http://localhost:8000/auth/callback"

# Additional URLs derived from the provider
AUTHORIZATION_URL = f"{OIDC_PROVIDER_URL}/o/oauth2/auth"
TOKEN_URL = f"{OIDC_PROVIDER_URL}/o/oauth2/token"
USERINFO_URL = f"{OIDC_PROVIDER_URL}/oauth2/v3/userinfo"


3. **Create FastAPI App with Authentication Routes**  
   Set up the FastAPI application with routes for login, callback, and user information.


In [ ]:
# main.py

from fastapi import FastAPI, Depends, HTTPException, status
from fastapi.responses import RedirectResponse
from authlib.integrations.starlette_client import OAuth
from starlette.requests import Request
from starlette.config import Config

app = FastAPI()

# Configuration
config_data = {
    'OIDC_CLIENT_ID': 'your-client-id',
    'OIDC_CLIENT_SECRET': 'your-client-secret',
    'OIDC_REDIRECT_URI': 'http://localhost:8000/auth/callback'
}
config = Config(environ=config_data)

oauth = OAuth(config)
oauth.register(
    name='oidc',
    client_id=config_data['OIDC_CLIENT_ID'],
    client_secret=config_data['OIDC_CLIENT_SECRET'],
    authorize_url=AUTHORIZATION_URL,
    authorize_params=None,
    access_token_url=TOKEN_URL,
    access_token_params=None,
    refresh_token_url=None,
    redirect_uri=config_data['OIDC_REDIRECT_URI'],
    client_kwargs={'scope': 'openid profile email'}
)

@app.get('/login')
async def login(request: Request):
    redirect_uri = config_data['OIDC_REDIRECT_URI']
    return await oauth.oidc.authorize_redirect(request, redirect_uri)

@app.route('/auth/callback')
async def auth_callback(request: Request):
    token = await oauth.oidc.authorize_access_token(request)
    user_info = await oauth.oidc.parse_id_token(request, token)
    
    # Do something with user_info, e.g., store it in session
    return {"user_info": user_info}

@app.get('/')
def read_root():
    return {"message": "Welcome to the OIDC demonstration"}

if __name__ == '__main__':
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)


4. **Add Security Dependencies**  
   Optionally, you can create security dependencies to ensure protected endpoints are only accessible to authenticated users.


In [ ]:
# security.py

from fastapi import Depends, HTTPException, status
from authlib.integrations.starlette_client import OAuth
from starlette.requests import Request
import json

oauth = OAuth()

def get_current_user(request: Request):
    if 'user' not in request.session:
        raise HTTPException(status_code=status.HTTP_401_UNAUTHORIZED, detail="Not authenticated")
    
    user = request.session['user']
    return json.loads(user)

@app.get("/protected")
async def protected_route(user: dict = Depends(get_current_user)):
    return {"message": f"Hello, {user['name']}"}


**Examples**

- **Login Route**  
  The `/login` route redirects the user to the OIDC provider's authorization endpoint.


In [ ]:
@app.get('/login')
async def login(request: Request):
    redirect_uri = config_data['OIDC_REDIRECT_URI']
    return await oauth.oidc.authorize_redirect(request, redirect_uri)


- **Callback Route**  
  The `/auth/callback` route handles the response from the OIDC provider and fetches the user's information.


In [ ]:
@app.route('/auth/callback')
async def auth_callback(request: Request):
    token = await oauth.oidc.authorize_access_token(request)
    user_info = await oauth.oidc.parse_id_token(request, token)
    
    # Do something with user_info, e.g., store it in session
    return {"user_info": user_info}


- **Protected Route**  
  An example of a protected route that only authenticated users can access.


In [ ]:
@app.get("/protected")
async def protected_route(user: dict = Depends(get_current_user)):
    return {"message": f"Hello, {user['name']}"}


**Conclusion**  
By following these steps, you can integrate OpenID Connect (OIDC) into your FastAPI application for secure user authentication. This provides a robust method for verifying user identities without handling sensitive credentials directly.


**Authorization**  
Authorization is the process of verifying what a user has access to. This can be managed by defining roles and permissions within your application.


In [ ]:
def get_current_active_user(current_user: dict = Depends(get_current_user)):
    if current_user.get("disabled"):
        raise HTTPException(status_code=400, detail="Inactive user")
    return current_user

def get_admin_user(current_user: dict = Depends(get_current_active_user)):
    if current_user.get("role") != "admin":
        raise HTTPException(status_code=403, detail="Not enough permissions")
    return current_user


**Dependency Injection**  
FastAPI's dependency injection system allows you to manage security dependencies effectively. You can encapsulate complex security logic within dependency functions.


In [ ]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/items/")
async def read_items(current_user: dict = Depends(get_current_active_user)):
    return [{"item_id": "Foo", "owner": current_user["username"]}]


**Advanced Features**

- **HTTPS**  
  Always use HTTPS in production to encrypt data in transit and protect it from eavesdropping.

- **CORS**  
  Enable Cross-Origin Resource Sharing (CORS) if your frontend is hosted on a different domain than your FastAPI backend.


In [ ]:
from fastapi.middleware.cors import CORSMiddleware

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


**Rate Limiting**  
Implement rate limiting to prevent abuse and enhance application security.

**Security Headers**  
Set appropriate security headers to mitigate common web vulnerabilities like XSS, CSRF, etc.

**Web Security**  
For traditional HTML content:
- **CSRF Protection:** Protect against Cross-Site Request Forgery attacks.
- **XSS Protection:** Sanitize and validate input to prevent Cross-Site Scripting attacks.
- **Session Management:** Securely manage sessions, often via cookies.

**Application Security**  
For API endpoints:
- **Input Validation:** Always validate input parameters.
- **SQL Injection Prevention:** Use ORMs or parameterized queries to prevent SQL Injection.
- **Rate Limiting:** Prevent abuse by rate-limiting requests.
- **Auditing and Logging:** Keep logs to track access and identify potential breaches.

**Example Setup**  
An example FastAPI setup combining these elements:


In [ ]:
from fastapi import FastAPI, Depends, HTTPException
from fastapi.security import OAuth2PasswordBearer
from starlette.middleware.cors import CORSMiddleware
import jwt

app = FastAPI()

# OAuth2 scheme
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

# CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

SECRET_KEY = "your_secret_key"
ALGORITHM = "HS256"

async def get_current_user(token: str = Depends(oauth2_scheme)):
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        username = payload.get("sub")
        if username is None:
            raise HTTPException(status_code=401, detail="Invalid credentials")
    except jwt.PyJWTError:
        raise HTTPException(status_code=401, detail="Invalid credentials")
    return {"username": username}

@app.get("/items/")
async def read_items(current_user: dict = Depends(get_current_user)):
    return [{"item_id": "Foo", "owner": current_user["username"]}]


In conclusion, FastAPI provides a comprehensive set of tools and practices for implementing security at both web and application levels, enabling you to build secure APIs and web applications effectively.

*******************************************************************
# Simple and Some Advanced Securities

**Simple and Useful Security Specifications**

- **Input Validation**  
  Ensure all inputs are validated to prevent injection attacks, such as SQL injection and cross-site scripting (XSS).


In [ ]:
from pydantic import BaseModel, constr

class Item(BaseModel):
    name: constr(min_length=1, max_length=100)
    description: Optional[str] = None

@app.post("/items/")
async def create_item(item: Item):
    return item


- **CSRF Protection**  
  Mitigate Cross-Site Request Forgery (CSRF) by using anti-CSRF tokens in your forms and API requests.

- **Secure Headers**  
  Use secure headers like Content Security Policy (CSP), X-Frame-Options, and others to protect against common vulnerabilities.


In [ ]:
from starlette.middleware.httpsredirect import HTTPSRedirectMiddleware

app.add_middleware(
    HTTPSRedirectMiddleware
)


**Logging and Monitoring**  
Implement logging and monitoring to detect suspicious activities in real-time.


In [ ]:
import logging

logging.basicConfig(filename='app.log', level=logging.INFO)

@app.get("/log-example")
async def log_example():
    logging.info("This is an info log")
    return {"message": "Check your logs"}


**Advanced Password Security Management**  
Effective password security is crucial for protecting user accounts. Here are some advanced password security management techniques:

- **Password Hashing**  
  Store passwords as hashes rather than plain text. Use strong hashing algorithms like bcrypt.


In [ ]:
from passlib.context import CryptContext

pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

def hash_password(password: str) -> str:
    return pwd_context.hash(password)

def verify_password(plain_password: str, hashed_password: str) -> bool:
    return pwd_context.verify(plain_password, hashed_password)


- **Password Salting**  
  Combine user passwords with unique salts before hashing to preclude rainbow table attacks.


In [ ]:
import os

def generate_salt() -> str:
    return os.urandom(32).hex()

def hash_password_with_salt(password: str, salt: str) -> str:
    combined = password + salt
    return pwd_context.hash(combined)


- **Enforce Strong Passwords**  
  Enforce policies that require users to use complex passwords.


In [ ]:
from pydantic import BaseModel, Field, validator

class UserCreate(BaseModel):
    username: str
    password: str

    @validator('password')
    def password_complexity(cls, value):
        if len(value) < 8 or not any(c.isdigit() for c in value) or not any(c.isalpha() for c in value):
            raise ValueError('Password must be at least 8 characters long and include both letters and numbers')
        return value


- **Multi-Factor Authentication (MFA)**  
  Add an extra layer of security by requiring two or more verification methods.
.

In [ ]:
# Example setup for Time-based One-Time Password (TOTP)
# This example uses the pyotp library
import pyotp

# Generate a TOTP secret key
totp_secret = pyotp.random_base32()

# Generate a TOTP based on the current time
totp = pyotp.TOTP(totp_secret)
print(f"Current TOTP: {totp.now()}")

# Verify the provided TOTP
def verify_totp(totp_provided: str, secret: str) -> bool:
    totp = pyotp.TOTP(secret)
    return totp.verify(totp_provided)


**Emerging Technologies in Security**

- **Zero Trust Architecture**  
  Zero Trust is a security model centered around the principle of never trust, always verify. It assumes no one is trusted by default, inside or outside the network.

- **Homomorphic Encryption**  
  Homomorphic encryption allows computations to be performed directly on encrypted data without needing access to a secret key, thus preserving privacy.

- **Blockchain for Security**  
  Blockchain technology can provide decentralized and tamper-resistant ledgers for improved security in various applications, from supply chain management to digital identity verification.

- **AI and Machine Learning**  
  AI and machine learning are increasingly used to identify patterns and detect anomalies in real-time, enhancing threat detection and response capabilities.

- **Quantum Cryptography**  
  Quantum cryptography leverages the principles of quantum mechanics to create theoretically unbreakable encryption.

- **Secure Access Service Edge (SASE)**  
  SASE combines wide-area networking and security into a single cloud-delivered service model to enhance secure access to resources regardless of location.

**Conclusion**  
By combining these simple and advanced security measures, you can significantly enhance the security posture of your web and application architectures. Understanding and implementing cutting-edge technologies will further future-proof your systems against evolving security threats.
